### Check the transient response of the stereo widener with percussive signals

In [ ]:
import numpy as np
import soundfile as sf
import os, sys
from typing import Optional
import matplotlib.pyplot as plt
from scipy.signal import spectrogram
from importlib import reload
path = os.path.abspath('../.')
sys.path.append(path)

In [ ]:
from widener import *
from onset_detector import OnsetDetector
from utils import db, ms_to_samps
from plot import *
save_flag = True

#### Helper function

In [ ]:
def plot_spectrogram(signal: np.ndarray, fs:float, title: str, nfft:int=2**10, axis:Optional[int]=None):
    if axis is None:
        ff, tt, Sxx = spectrogram(signal, fs=fs, window='hann', nfft=nfft)
    else:
        ff, tt, Sxx = spectrogram(signal, fs=fs, window='hann', nfft=nfft, axis=axis)
    Sxx_norm = Sxx/np.max(np.abs(Sxx))
    Sxx_db = db(np.abs(Sxx_norm))
    if Sxx.ndim == 3:    
        fig, ax = plt.subplots(2,1)
        for num_sig in range(Sxx.shape[1]):
            ax[num_sig].imshow(np.squeeze(Sxx_db[:, num_sig, :]), 
                               cmap = 'jet', 
                               extent=[0,max(tt),0,max(ff)], 
                               interpolation='bilinear', 
                               aspect = 'auto', 
                               origin = 'lower')
            ax[num_sig].set_xlabel('Time (s)') 
            ax[num_sig].set_ylabel('Frequency (Hz)')
            ax[num_sig].set_ylim([20,20000])
            ax[num_sig].grid(True)
        ax[0].set_title(title)

    else:
        fig, ax = plt.subplots()
        ax.imshow(np.squeeze(Sxx_db), 
                   cmap = 'jet', 
                   extent=[0,max(tt),0,max(ff)], 
                   interpolation='bilinear', 
                   aspect = 'auto', 
                   origin = 'lower')
        ax.set_title(title)
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Frequency (Hz)')
        ax.set_ylim([20,20000])
        ax.grid(True)
    return fig

#### Check the onset detector

In [ ]:
input_signal, fs = sf.read('../../audio/lexicon_dry15.wav')
trim_start_samp = int(ms_to_samps(1200, fs))
trim_end_samp = int(ms_to_samps(3000, fs))
trimmed_input_signal = input_signal[trim_start_samp : trim_end_samp, :]
# plot spectrogram
fig = plot_spectrogram(trimmed_input_signal[:,0], fs, 'Input signal', axis=0, nfft=2**9)
if save_flag:
    fig.savefig('../../figures/transient_input_spec.png', dpi=300)

In [ ]:
onset_detect = OnsetDetector(fs, 
                             attack_time_ms=5.0, 
                             release_time_ms=20.0,
                             min_onset_hold_ms=80.0,
                             min_onset_sep_ms=20.0)
onset_detect.process(input_signal, to_plot=True)

#### Broadband widener with and without transient detection

In [ ]:
decorr_type = DecorrelationType.ALLPASS
beta_init = np.pi/2
stereo_widener = StereoWidenerBroadband(input_signal, fs, decorr_type, beta_init)
output_signal = stereo_widener.process()

# plot spectrogram
trimmed_output_signal = output_signal[trim_start_samp : trim_end_samp, :]
fig = plot_spectrogram(trimmed_output_signal[:,0], fs, 'Broadband widener output', axis=0, nfft=2**9)
if save_flag:
    fig.savefig('../../figures/transient_output_spec_wo_detection.png', dpi=300)
    sf.write(f'../../audio/lexicon_broadband_beta={beta_init/(np.pi/2)}.wav', output_signal, fs)


In [ ]:
del stereo_widener
onset_detection_params = {
    'min_onset_hold_ms':80.0,
    'min_onset_sep_ms':20.0,
}
stereo_widener = StereoWidenerBroadband(input_signal, 
                                        fs, 
                                        decorr_type, 
                                        beta_init, 
                                        detect_transient=True, 
                                        onset_detection_params=onset_detection_params)
output_signal = stereo_widener.process()
trimmed_output_signal = output_signal[trim_start_samp : trim_end_samp, :]

# plot spectrogram
fig = plot_spectrogram(trimmed_output_signal[:,0], fs, 'Broadband widener output', axis=0, nfft=2**9)
if save_flag:
    fig.savefig('../../figures/transient_output_spec_with_detection.png', dpi=300)
    sf.write(f'../../audio/lexicon_broadband_transient_beta={beta_init/(np.pi/2)}.wav', output_signal, fs)

In [ ]:
from plot import time_plot
plt.figure()
plt.subplot(211)
time_plot(input_signal, fs, db_scale=False)
plt.subplot(212)
time_plot(output_signal, fs, db_scale=False)